# HW2

We first go over the data extraction part for both exercises. Then we show how we analyze the extracted data.

## Data Extraction

The following code shows the import statements as well as the links shared between data extraction for both bachelor  and master students:   

In [1]:
import numpy as np
import pandas as pd
import sys
from bs4 import BeautifulSoup as BSoup
import requests
formLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
showLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247"
r = requests.get(formLink)
soup = BSoup(r.text, "lxml")
# The following function generates the link to the corresponding HTML page for a particular field of study, 
# academic year, and semester.
def showLinkGen(htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV):
  return "{}&{}={}&{}={}&{}={}&{}={}".format(showLink, htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV)
selectFields = soup.find_all("select")
infoField=selectFields[0].find("option", text="Informatique")["value"]
infoFieldParam=selectFields[0]["name"]
semParam=selectFields[2]["name"]
allYears = [y["value"] for y in selectFields[1].find_all("option")[1:]]
yearParam = selectFields[1]["name"]
htmlradiobutton=soup.find("input", type="radio")
# For a given semester and year, returns the HTML page containing the list of the students' information.
def get_html(sem, year):
    link = showLinkGen(htmlradiobutton["name"], htmlradiobutton["value"], infoFieldParam, infoField, yearParam, 
                 year, semParam, sem)
    return requests.get(link)

### Bachelor Students' Data Extraction

Assumption
----------
We assume that all bachelor students finish in their 6th semester -- there is no bachelor student who finishes during 5th semester.

The following code extracts the parameters which should be passed to the previous function (`showLinkGen`) for all bachelor students of Computer Science.

In [2]:
bsSem1=selectFields[2].find("option", text="Bachelor semestre 1")["value"]
bsSem6=selectFields[2].find("option", text="Bachelor semestre 6")["value"]

The following function converts an HTML page containing the relevant part of the list of the students' information to a DataFrame.

In [3]:
def get_bs_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font").text.split(', ')
  semester=titleinfo[2]
  semester=int(semester[(len(semester)-2):])
  year=titleinfo[1]
  all_data=[]
  for elem in elems:
    items=elem.find_all("td")
    gender = "M" if (items[0].text == "Monsieur") else "F"
    sciper = int(items[10].text)
    all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester})
  return pd.DataFrame(all_data)

The following function uses the previously defined functions and returns an aggregated DataFrame of all bachelor students' information.

In [4]:
def get_bs_alldata():
  all_data = []
  for bsSem in [bsSem1, bsSem6]:
    for year in allYears:
        all_data.append(get_bs_dataframe(get_html(bsSem, year)))
  return pd.concat(all_data)

### Master Students' Data Extraction

Assumption
----------
Master students are considered to finish their studies only if they are registered for at least one master project.

The following parameters are used for generating the HTML link for all master students.

In [5]:
msSem1_text="Master semestre 1"
msSemPF_text="Projet Master automne"
msSemPS_text="Projet Master printemps"
msSem1=selectFields[2].find("option", text=msSem1_text)["value"]
msSemPF=selectFields[2].find("option", text=msSemPF_text)["value"]
msSemPS=selectFields[2].find("option", text=msSemPS_text)["value"]

The following function converts the given HTML page to a DataFrame containing the relevant information about a master student.

In [6]:
def get_ms_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font")
  if(titleinfo is None):
      return pd.DataFrame([])
  else:
      titleinfo = titleinfo.text.split(', ')
      semester=titleinfo[2]
      semester=1 if semester==msSem1_text else (2 if semester==msSemPF_text else 3)
      year=titleinfo[1]
      all_data=[]
      for elem in elems:
        items=elem.find_all("td")
        gender = "M" if (items[0].text == "Monsieur") else "F"
        sciper = int(items[10].text)
        spec = items[4].text <> ""
        minor = items[6].text <> ""
        all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester,
                        "Minor": minor, "Specialization": spec})
      return pd.DataFrame(all_data)

The following function aggregates the DataFrames for all master students. 

In [7]:
def get_ms_alldata():
  all_data = []
  for sem in [msSem1, msSemPF, msSemPS]:
    for year in allYears:
        all_data.append(get_ms_dataframe(get_html(sem, year)))
  return pd.concat(all_data)